In [8]:
import os
import zipfile
os.chdir('/project_data/data_asset')

In [9]:
%%writefile model_template.py
from docplex.mp.model import Model
from docplex.util.environment import get_environment
import numpy as np
import datetime as dt
import re
import json
from os.path import splitext
import pandas
from six import iteritems
from collections import namedtuple
import os
import sys

Claim = namedtuple("Claim", ["name", "complexity", "cost", "type"])
Handler = namedtuple("Handler", ["name", "skill", "load"])

def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")

def get_model(data, **kwargs):
    model = None
    if "handler_budget" in kwargs:
        model = _get_base_model(data, **kwargs)
    if "max_imbalance" in kwargs:
        model = _get_balanced_model(data, model=model, **kwargs)
    if "skill_level_diff" in kwargs:
        model = _get_sl_model(data, model=model, **kwargs)
    if "skill_restriction" in kwargs:
        model = _get_sr_model(data, model=model, **kwargs)

    return model

def _get_base_model(data, handler_budget=500, **kwargs):
    if len(data) != 3:
        raise ValueError('Data should be a list/tuple of 3 dataframes (claims, handlers, and leakages)')
        
    # Initialialize Model
    results = Model(**kwargs)
    
    # Unpack Data
    claims, handlers, leakages = data
    
    # --- decision variables ---
    results.ch_vars = results.binary_var_matrix(claims, handlers, "Assigned")

    # --- constraints ---
    # 1 claim to 1 handler
    for c in claims:
        results.add_constraint(1 == results.sum([results.ch_vars[c,h] for h in handlers]))

    # Claims assigned < handler budget
    for h in handlers:
        results.add_constraint(handler_budget >= results.sum([results.ch_vars[c,h] for c in claims]) + h.load) 
        
    # --- objective ---
    results.total_leakage_cost = results.sum([results.ch_vars[c,h] * leakages[c.name][h.name] * c.cost for c in claims for h in handlers])
    results.add_kpi(results.total_leakage_cost, 'Total Leakage Cost')

    results.minimize(results.total_leakage_cost)
    
    return results

def _get_balanced_model(data, max_imbalance=1, model=None, **kwargs):
    if model is None:
        results = _get_base_model(data, **kwargs)
    else:
        results = model
    # Unpack Data
    claims, handlers, leakages = data

    # --- constraints ---
    # Workload Balancing
    results.add_constraint(max_imbalance >= results.max([results.sum([results.ch_vars[c,h] for c in claims]) for h in handlers]) - results.min([results.sum([results.ch_vars[c,h] for c in claims]) for h in handlers]) )

    return results

def _get_sl_model(data, skill_level_diff=1, model=None, **kwargs):
    if model is None:
        results = _get_base_model(data, **kwargs)
    else:
        results = model
    # Unpack Data
    claims, handlers, leakages = data

    # --- constraints ---
    # skill level constraint
    for h in handlers:
        results.add_constraint( skill_level_diff >= results.max([results.ch_vars[c,h]*(c.complexity - h.skill) for c in claims]) )

    return results

def _get_sr_model(data, skill_restriction=1, model=None, **kwargs):
    if model is None:
        results = _get_base_model(data, **kwargs)
    else:
        results = model
    # Unpack Data
    claims, handlers, leakages = data

    # --- constraints ---
    # skill limit constraint
    for h in handlers:
        results.add_constraint(skill_restriction >= results.sum([results.ch_vars[c,h]*results.max([np.sign(c.complexity - h.skill),0]) for c in claims]) )

    return results

def process_data(data, claim, handler, leakage):
    try:
        df_claims, df_handlers, df_leakage = data
    except:
        # Seperate data into dataframes
        df_claims = data[claim].drop_duplicates(subset=claim[0]).reset_index(drop=True)
        df_handlers = data[handler].drop_duplicates(subset=handler[0]).reset_index(drop=True)
        df_leakage = data[[claim[0]] + [handler[0]] + leakage].reset_index(drop=True)
    
    # Pre-allocate row memory
    handlers = [None]*df_handlers.shape[0]
    claims = [None]*df_claims.shape[0]
    
    # Convert handlers and claims to list of named tuples
    for row_idx, row in df_handlers.iterrows():
        handlers[row_idx] = Handler(*list(row))
    
    for row_idx, row in df_claims.iterrows():
        claims[row_idx] = Claim(*list(row))
    
    # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
        # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
    leakages = {c_i.name:{} for c_i in claims}
    try:
        for row in range(df_leakage.shape[0]): 
            leakages[df_leakage.iloc[row,:]["claim"]][df_leakage.iloc[row,:]["handler"]] = df_leakage.iloc[row,:]["leakage_cost"]
    except:
        for row in range(df_leakage.shape[0]): 
            leakages[df_leakage.iloc[row,:][claim[0]]][df_leakage.iloc[row,:][handler[0]]] = df_leakage.iloc[row,:][leakage[0]] 
    
    return claims, handlers, leakages
            
class ClaimsLeakageModel():
    def __init__(self, **kwargs):
        self.model_args = kwargs

    def build_model(self, data):
        model = get_model(data, **self.model_args)
        self.model = model

    def solve(self):
        self.s = self.model.solve()
        if self.s: # Model is able to be solved
            self._parse_solution()
        else: # No feasible solution
            raise ValueError("No feasible solution")

    def _parse_solution(self):
        '''
        TODO: Fix regex parsing to only look for things after name= and not jsut anything between two single quotes
        '''
        json_solution = json.loads(self.model.solution.export_as_json_string())['CPLEXSolution']['variables']
        regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
        file_timestamp = dt.datetime.now().strftime(format="%d%m%y_%H%M%S")
        with open("solution_{}.csv".format(file_timestamp), "w+") as fp:
            fp.write("claim_id,handler_id\n")
            for element in json_solution:
                '''
                TODO: Parse name to get claim id and handler name and the assigned value
                '''
                try:
                    claim_id, handler_id = regex_get_id.findall(element['name'])
                except:
                    claim_id, _, handler_id = regex_get_id.findall(element['name'])
                fp.write("{},{}\n".format(claim_id, handler_id))


    
if __name__ == "__main__":
    # Load CSV files into inputs dictionnary
    inputs = get_all_inputs()

    if len(inputs) == 2:
        df = inputs['data']

        handlers = ['Handler Allocation', 'Claim Handler Level', 'Handler Case Load at Allocation']
        claims = ['CLAIM_NBR', 'Claim Complexity', "Base Claims Cost", 'Claim Type']
        leakage = ['Pred_Leakage']
        claims, handlers, leakages = process_data(df, claims, handlers, leakage)
    else:
        Claim = namedtuple("Claim", ["name", "complexity", "cost","type"])
        Handler = namedtuple("Handler", ["name", "load", "skill"])
        df_claims, df_handlers, df_leakage = inputs["claim"], inputs["handler"], inputs["leakage"]
        handlers = ['Handler Allocation', 'Claim Handler Level', 'Handler Case Load at Allocation']
        claims = ['CLAIM_NBR', 'Claim Complexity', "Base Claims Cost", 'Claim Type']
        leakage = ['Pred_Leakage']

        claims, handlers, leakages = process_data([df_claims, df_handlers, df_leakage], claims, handlers, leakage)

    # return claims handler leakage list needed for model builing
    params = inputs['params'].to_dict("records")[0]

    model = get_model((claims, handlers, leakages), **params)

    ok = model.solve()

    solution_df = pandas.DataFrame(columns=['name', 'value'])
    regex_get_id= re.compile("'(\w+)'", re.IGNORECASE)
    for index, element in enumerate(json.loads(model.solution.export_as_json_string())['CPLEXSolution']['variables']):
        try:
            claim_id, handler_id = regex_get_id.findall(element['name'])
        except:
            claim_id, _, handler_id = regex_get_id.findall(element['name'])
        solution_df.loc[index,'name'] = claim_id
        solution_df.loc[index,'value'] = handler_id

    outputs = {}
    outputs['solution'] = solution_df

    # Generate output files
    write_all_outputs(outputs)    

Overwriting model_template.py


In [10]:
zf = zipfile.ZipFile('model_template.zip', mode='w')
try:
    print('adding model_template.zip')
    zf.write('model_template.py')
finally:
    print('closing')
    zf.close()

adding model_template.zip
closing


In [11]:
%%writefile test_model_template.py
import pytest
import numpy as np
import model_template as mt
import pandas as pd
from collections import namedtuple
import copy

Claim = namedtuple("Claim", ["name", "complexity", "cost"])
Handler = namedtuple("Handler", ["name", "load", "skill"])

@pytest.fixture(scope="module")
def mybase3model():
    return mt.ClaimsLeakageModel(handler_budget=3)

@pytest.fixture(scope="module")
def mybase5model():
    return mt.ClaimsLeakageModel(handler_budget=5)

@pytest.fixture(scope="module")
def mybalancedmodel():
    return mt.ClaimsLeakageModel(handler_budget=5, max_imbalance=1)

@pytest.fixture(scope="module")
def myslmodel():
    return mt.ClaimsLeakageModel(handler_budget=3, skill_level_diff=2)

@pytest.fixture(scope="module")
def mysrmodel():
    return mt.ClaimsLeakageModel(handler_budget=3, skill_restriction=2)

@pytest.fixture(scope="module")
def mytestdata():
    # read in data
    df_claims = pd.read_csv("../data/test_data/claim.csv")
    df_handlers = pd.read_csv("../data/test_data/handler.csv")
    df_leakage = pd.read_csv("../data/test_data/leakage.csv")
    # process data
    handlers = [None]*df_handlers.shape[0]
    claims = [None]*df_claims.shape[0]
    # Convert handlers and claims to list of named tuples
    for row_idx, row in df_handlers.iterrows():
        handlers[row_idx] = Handler(*list(row))

    for row_idx, row in df_claims.iterrows():
        claims[row_idx] = Claim(*list(row))

    # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
    leakages = {c_i.name:{} for c_i in claims}
    for row in range(df_leakage.shape[0]): 
        leakages[df_leakage.iloc[row,:]["claim"]][df_leakage.iloc[row,:]["handler"]] = df_leakage.iloc[row,:]["leakage_cost"] 
    
    # return claims handler leakage list needed for model builing
    return claims, handlers, leakages

@pytest.fixture(scope="module")
def mytestdata2():
    # read in data
    df_claims = pd.read_csv("../data/test_data/claim.csv")
    df_handlers = pd.read_csv("../data/test_data/handler.csv")
    df_leakage = pd.read_csv("../data/test_data/leakage.csv")
    # process data
    handlers = [None]*df_handlers.shape[0]
    claims = [None]*df_claims.shape[0]
    # Convert handlers and claims to list of named tuples
    for row_idx, row in df_handlers.iterrows():
        handlers[row_idx] = Handler(*list(row))

    for row_idx, row in df_claims.iterrows():
        claims[row_idx] = Claim(*list(row))

    # Convert leakages into a dictionary(claims:dictionary(handler:leakage))
    leakages = {c_i.name:{} for c_i in claims}
    for row in range(df_leakage.shape[0]): 
        leakages[df_leakage.iloc[row,:]["claim"]][df_leakage.iloc[row,:]["handler"]] = df_leakage.iloc[row,:]["leakage_cost"] 
    
    # return claims handler leakage list needed for model builing
    return claims, handlers, leakages

def teardown_module(module):
    import os
    filenames = os.listdir(os.getcwd())
    for fname in filenames:
        if fname.endswith('.csv'):
            os.remove(fname)

### TESTS ###

def test_input_data_claims(mytestdata):
    claims, handlers, leakages = mytestdata
    assert len(claims) == 5

def test_input_data_handlers(mytestdata):
    claims, handlers, leakages = mytestdata
    assert len(handlers) == 3

def test_input_data_leakage(mytestdata): 
    claims, handlers, leakages = mytestdata
    assert len(leakages) == 5

def test_base3_model(mytestdata, mybase3model): # Objective should equal 45
    claims, handlers, leakages = mytestdata
    mybase3model.build_model((claims, handlers, leakages))
    s = mybase3model.model.solve()
    assert s._objective == 45

def test_base5_model(mytestdata, mybase5model): # Objective should equal 28
    claims, handlers, leakages = mytestdata
    mybase5model.build_model((claims, handlers, leakages))
    s = mybase5model.model.solve()
    assert s._objective == 30

def test_balanced_model(mytestdata, mybalancedmodel): # Objective should equal 35
    claims, handlers, leakages = mytestdata
    mybalancedmodel.build_model((claims, handlers, leakages))
    s = mybalancedmodel.model.solve()
    assert s._objective == 35

def test_sl_model(mytestdata2, myslmodel): # Objective should equal 35
    claims, handlers, leakages = mytestdata2
    myslmodel.build_model((claims, handlers, leakages))
    s = myslmodel.model.solve()
    assert s._objective == 32

def test_sr_model(mytestdata2, mysrmodel): # Objective should equal 35
    claims, handlers, leakages = mytestdata2
    mysrmodel.build_model((claims, handlers, leakages))
    s = mysrmodel.model.solve()
    assert s._objective == 14

Overwriting test_model_template.py
